In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.random.seed(0)
import pickle
#plt.style.use("ggplot")
import tensorflow as tf

##Data preprocessing 

In [2]:
data = pd.read_csv('Actual_15_Validation_Data_80.csv', delimiter=';')
#data = data.fillna(method = 'ffill')
data.head()

,Sentence #,Word,Predicted Tag
0,1,Such,NaN
1,1,systems,NaN
2,1,can,NaN
3,1,make,NaN
4,1,earlier,NaN


In [3]:
data['Sentence #'].value_counts()

Sentence #
59    76
44    59
65    59
25    55
28    53
      ..
13    12
48    11
29    11
6     11
7      9
Name: count, Length: 75, dtype: int64

In [4]:
print('unique words in corpus:', data['Word'].nunique())

unique words in corpus: 840


In [5]:
class SentenceGetter(object):
    """Retrieves sentences and their corresponding tags."""
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        aggregate_function = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(),
                                                                    s['Predicted Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(aggregate_function)
        self.sentences = [s for s in self.grouped]

In [6]:
getter = SentenceGetter(data)

In [7]:
sentences = getter.sentences

In [8]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

### Loading our tag to index from save file for our new dataset

In [9]:
with open('word_to_indexCombinedDataForTest.pkl', 'rb') as f:
    word_to_index = pickle.load(f)

In [10]:
with open('tag_to_indexCombinedDataForTest.pkl', 'rb') as f:
    tag_to_index = pickle.load(f)

##Getting the new words and tags for new dataset

In [11]:
words = [k for k in word_to_index.keys()]
num_words = len(words)

In [12]:
tags = [k for k in tag_to_index.keys()]
num_tags = len(tags)

In [13]:
print(word_to_index)

{'per': 1, 'fires': 2, 'owing': 3, 'Malware': 4, 'fine': 5, 'purpose\u200d—\u200csuch': 6, 'results': 7, 'study': 8, 'factors': 9, 'Diamante': 10, 'moreover': 11, '1,000,000,000,000': 12, 'Inputs': 13, 'Beneath': 14, 'S-100': 15, 'uncorrected': 16, 'integrate': 17, 'Breather': 18, 'mixed': 19, 'came': 20, 'update': 21, '80387': 22, 'beneath': 23, 'elliptic': 24, 'actuator': 25, 'Hexagon': 26, 'downshifting': 27, 'overlapping': 28, 'encounters': 29, 'temporarily': 30, 'automotive': 31, 'hung': 32, 'one': 33, 'quantities': 34, 'LDPC': 35, 'stopped': 36, 'limits': 37, 'synchronisation': 38, 'durability': 39, 'positive': 40, 'mix': 41, 'perf': 42, '2^{M': 43, 'pH': 44, 'continuous': 45, 'transistor': 46, 'Model': 47, '14': 48, 'having': 49, 'capacitor': 50, 'uncorrectable': 51, 'organize': 52, 'semiconductor': 53, 'maps': 54, 'such': 55, 'AMD': 56, 'connections': 57, 'Masatoshi': 58, 'multiplier': 59, 'x4': 60, 'Application': 61, 'complement': 62, 'comparators': 63, 'megabyte': 64, 'previo

In [14]:
print(tag_to_index)

{'I-qt': 0, 'B-hwp': 1, 'B-sw': 2, 'B-qt': 3, 'B-hwc': 4, 'B-comp': 5, 'B-func': 6, 'I-mea': 7, 'O': 8, 'I-func': 9, 'I-sw': 10, 'I-hwp': 11, 'I-unit': 12, 'I-sys': 13, 'I-hwc': 14, 'B-sys': 15, 'I-comp': 16, 'I-hwsp': 17, 'B-unit': 18, 'B-mea': 19, 'B-hwsp': 20}


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50

* Taking sentences from our new dataset and convert each of the words of the sentences to our trained index. as we trained our model with index number we have to give index number again when predictiong new data.

#preparing our test dataset X

In [16]:
X = [[word_to_index[w[0]] for w in s if w[0] in word_to_index.keys()] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)
#we are not using any tag here
# y = [[tag2index[w[2]] for w in s] for s in sentences]
# y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2index["O"])
# y = [to_categorical(i, num_classes=num_tags) for i in y]

> same shape as x_train of our model

In [17]:
X.shape

(75, 50)

In [18]:
from tensorflow.keras.models import load_model
new_model = load_model('KB_BI_LSTM_model_rm_80.h5')

In [19]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 50)            320400    
                                                                 
 spatial_dropout1d (Spatial  (None, 50, 50)            0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 50, 512)           628736    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 50, 21)            10773     
 ributed)                                                        
                                                             

## Predicting new data with our trained model. will pass X[i] as our "data to be predicted" which is basically index number. then convert it to word using for loop.

In [20]:
list_words = []
list_tags_true =[]
list_tags_pred =[]

for i in range(X.shape[0]):
  preds = new_model.predict(np.array([X[i]])) ##passing indexes of our new data
  pred = np.argmax(preds, axis=-1)
  for w,p in zip(X[i],pred[0]):
    list_words.append(words[w-1])
    list_tags_pred.append(tags[p])
table = pd.DataFrame({'Word':list_words,'Pred':list_tags_pred})

#print(table.to_string())
table.to_csv('LSTM_Evaluation_Result_on_rm_80.csv')

1/1 [==============================] - 0s 26ms/step
